In [8]:
import pandas
import numpy

df = pandas.read_csv('../data/FIRE_STARTS_PER_ZONE.csv')
df['IGNITION_DATE'] = df['IGNITION_DATE'].map(pandas.to_datetime)
df['TIMESTAMP'] = df['IGNITION_DATE'].values.astype(numpy.int64) # 10 ** 9
df



,Unnamed: 0,FIRE_CENTRE,ZONE,IGNITION_DATE,COUNT,TIMESTAMP
0,0,2.0,0.0,1950-05-01,1,-620784000000000000
1,1,2.0,0.0,1950-05-04,1,-620524800000000000
2,2,2.0,0.0,1950-05-12,1,-619833600000000000
3,3,2.0,0.0,1950-05-14,2,-619660800000000000
4,4,2.0,0.0,1950-05-16,7,-619488000000000000
...,...,...,...,...,...,...
81303,81303,7.0,24.0,1973-09-08,2,116294400000000000
81304,81304,7.0,24.0,1973-09-11,1,116553600000000000
81305,81305,7.0,24.0,1973-09-16,1,116985600000000000
81306,81306,7.0,24.0,1973-09-19,1,117244800000000000


In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std

X=df[['FIRE_CENTRE', 'ZONE', 'TIMESTAMP']]  # Features
y=df['COUNT']  # Labels

# create regression model
model=RandomForestRegressor(n_estimators=100)
# evaluate the model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X.values, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))


MAE: -1.190 (0.024)


In [17]:
model.fit(X.values, y)
# Predict into the future!
# Count of fires in fire centre 1, zone 1 on Sat May 27 2023 04:48:19 GMT+0000
out = model.predict([[1, 1, 1685162899]])
out

array([1.02])